In [3]:
!git config --global user.email "akiraroyal184@gmail.com"
!git config --global user.name "Akira Royal"
!git clone https://github.com/TechX-Resources/SignLanguageTranslator.git

Cloning into 'SignLanguageTranslator'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), 253.69 KiB | 5.40 MiB/s, done.


In [7]:
!ls




sample_data  SignLanguageTranslator


In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np

import the dataset

In [2]:
train_df = pd.read_csv("sign_mnist_train.csv")
test_df = pd.read_csv("sign_mnist_test.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'sign_mnist_train.csv'

Look at the train and test shape and look at the first 5 rows of the data

In [ ]:
print("Train Shape:",train_df.shape)
print("Test Shape:",test_df.shape)
print(train_df.head())

Get all the rows and columns starting at 1 for X_train AND
Get all the awnsers(labels) for y_train

In [ ]:
X_train = train_df.iloc[:, 1:].values
y_train = train_df.iloc[:, 0].values

Get all the rows and columns starting at 1 for X_test AND
Get all the awnsers(labels) for y_test

In [ ]:
X_test = test_df.iloc[:, 1:].values
y_test = test_df.iloc[:, 0].values

When you look at the data it shows 785 columns, minus the label its 784.The read me on kaggle shows its 28X28 pixels,greyscale

We use CNN for classifying images.(-1)auto detects the number of images, (28,28) is the pixels and (1) is used for greyscale

We then call X_train & X_test.shape to see its new shape

In [ ]:
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)

print('X_train shape: ', X_train.shape)
print('X_test shape: ', X_test.shape)

see what the data looks like

In [ ]:
plt.imshow(X_train[0].reshape(28, 28), cmap='gray')
plt.title(f"Label: {y_train[0]}")
plt.show()

Normalize Pixel values by turning each number to a float then dividing it by 225 (bc its the maximun pixel value for greyscale)so you get a range between 0 and 1


In [ ]:

X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0


to_categorial converts label numbersand  One Hot encodes it so the data is now in proper format for building the model

In [ ]:
from tensorflow.keras.utils import to_categorical
import numpy as np


unique_labels = np.unique(np.concatenate((y_train, y_test)))
label_map = {label: i for i, label in enumerate(unique_labels)}

y_train_remapped = np.array([label_map[label] for label in y_train])
y_test_remapped = np.array([label_map[label] for label in y_test])


num_classes = len(unique_labels)

y_train_cat = to_categorical(y_train_remapped, num_classes=num_classes)
y_test_cat = to_categorical(y_test_remapped, num_classes=num_classes)

Our next step is to build the model a CNN, which is the standard model for image classification. It works well with small grayscale images like ours, and it’s proven to work for similar datasets. We do this by using tensorflow



In [ ]:
class_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
               'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

plt.figure(figsize=(20,20))
for i in range(100):
    plt.subplot(10,10,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[i], cmap='gray')
    plt.xlabel(class_names[y_train[i]])
plt.show()

# Creating convolutional base

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='tanh', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='tanh'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='tanh'))

In [ ]:
model.summary()

# Adding Dense layers

In [ ]:
num_classes = len(np.unique(y_train))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='tanh'))
model.add(layers.Dense(num_classes))

In [ ]:
model.summary()

# Compiling and training model

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(X_train, y_train_cat, epochs=10,
                    validation_data=(X_test, y_test_cat))

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(X_test,  y_test_cat, verbose=2)